# Missing Value Handling Strategy & Feature Engineering
``` Preprocessing
Date: 11/04/2025
Goal: 
    - Age Missing Values (~20%): Imputed with the median (with an option for group-based imputation).
    - Deck Missing Values (High Volume): Due to a lack of clear information, candidates for handling include filling with 'Unknown' or dropping the feature entirely.
    - Embarked Missing Values (Minority): Imputed with the mode (the most frequent value).

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("../data/titanic_train.csv")
df.head()
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


(891, 12)

In [5]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,891.0,NaN,NaN,NaN,446.0,257.353842,1.0,223.5,446.0,668.5,891.0
Survived,891.0,NaN,NaN,NaN,0.383838,0.486592,0.0,0.0,0.0,1.0,1.0
Pclass,891.0,NaN,NaN,NaN,2.308642,0.836071,1.0,2.0,3.0,3.0,3.0
Name,891,891,"Dooley, Mr. Patrick",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,714.0,NaN,NaN,NaN,29.699118,14.526497,0.42,20.125,28.0,38.0,80.0
SibSp,891.0,NaN,NaN,NaN,0.523008,1.102743,0.0,0.0,0.0,1.0,8.0
Parch,891.0,NaN,NaN,NaN,0.381594,0.806057,0.0,0.0,0.0,0.0,6.0
Ticket,891,681,347082,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,891.0,NaN,NaN,NaN,32.204208,49.693429,0.0,7.9104,14.4542,31.0,512.3292


In [7]:
missing = df.isnull().sum().sort_values(ascending=False)
missing[missing>0]

Cabin       687
Age         177
Embarked      2
dtype: int64

In [23]:
def preprocess_basic(df):
    df = df.copy()
    # Set 'Title'
    df['Title'] = df['Name'].str.extract(r',\s*([^\.]+)\.')  # Mr, Mrs 추출
    df['Title'] = df['Title'].str.strip()
   
    # Age: fille NaN with median values of groups
    df['Age'] = df.groupby(['Pclass','Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
    
    # Embarked: fill with values from mode()
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    
    # FamilySize
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    return df

df2 = preprocess_basic(df)
df2.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Title            0
FamilySize       0
dtype: int64

In [24]:
# Comparing df and af2
before = df.isnull().sum()
after = df2.isnull().sum()
pd.DataFrame({'before':before,'after':after})

,before,after
Age,177.0,0
Cabin,687.0,687
Embarked,2.0,0
FamilySize,NaN,0
Fare,0.0,0
Name,0.0,0
Parch,0.0,0
PassengerId,0.0,0
Pclass,0.0,0
Sex,0.0,0
